In [3]:
import pandas as pd
from os import walk
import spacy
from tqdm import tqdm
# nlp = spacy.load("en_core_sci_scibert")

Fill your train path

In [4]:
my_paths = {
    'train': '../data/trainingdata_v3/train/',
    'dev': '../data/trainingdata_v3/dev/'
}
dataset_types = ['train', 'dev']

# Not Splitted

In [160]:
for dataset_type in dataset_types:
    filenames = next(walk(my_paths[dataset_type]), (None, None, []))[2]
    # print(filenames[:20])
    filenames_ann = [filename for filename in filenames if 'ann' in filename]
    filenames_txt = [filename for filename in filenames if 'txt' in filename]
    filenames_ann.sort()
    filenames_txt.sort()
    filenames_ann_txt = list(zip(filenames_ann, filenames_txt))

    df_dataset = pd.DataFrame()
    for index, (filename_ann, filename_txt) in enumerate(filenames_ann_txt):
        # print('Current index:', index)
        df_ann = pd.read_csv(my_paths[dataset_type] + filename_ann, sep='\t', names=['entity-event-context', 'classification-type', 'value'])
        df_entities = pd.DataFrame()
        # print(filename_ann)
        if (df_ann.apply(lambda row: row['value'] if row['entity-event-context'][0]== 'T' else '', axis=1).shape[0] > 0):
            df_entities['entities'] = df_ann.apply(lambda row: row['value'].lower() if row['entity-event-context'][0]== 'T' else '', axis=1)
            df_entities = df_entities[df_entities['entities'] != '']
            df_entities = df_entities.drop_duplicates(['entities'])
            entitites =  ','.join(df_entities.entities)
        else:
            entitites = ''
        with open(my_paths[dataset_type] + filename_txt, 'r') as file:
            text = file.readlines()
        text = ''.join(text).replace('\n', ' ').replace('\t', ' ')
        text_filtered = ''
        for char_index, char in enumerate(text):
            if char_index < (len(text)-1):
                if not(text[char_index + 1] == ' ' and char == ' '):
                    text_filtered += char
        text_filtered = text_filtered.lower()
        row = pd.DataFrame({ 'filename': filename_ann[:-4], 'text': [text_filtered], 'entities': [entitites]})
        df_dataset = pd.concat([df_dataset, row], ignore_index=True)

    # print(df_dataset.head())
    df_dataset.to_csv(my_paths[dataset_type] + dataset_type + '_entities.tsv', sep='\t')




In [ ]:
from transformers import AutoTokenizer

In [4]:
df = pd.DataFrame()
for dataset_type in [dataset_types[0]]:
    df = pd.concat([df, pd.read_csv(my_paths[dataset_type] + dataset_type + '_entities.tsv', sep='\t')], ignore_index=True)

In [5]:
df = df[['filename', 'text', 'entities']]

In [3]:
from transformers import AutoTokenizer
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
df.loc[0, ['text']].values[0]

In [9]:
len(tokenizer.encode(df.loc[0, ['text']].values[0]))

593

In [10]:
df['tokenizer_length_text'] = df.apply(lambda row: len(tokenizer.encode(row['text'])) , axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1489 > 1024). Running this sequence through the model will result in indexing errors


In [17]:
df

,filename,text,entities,tokenizer_length_text,tokenizer_length_entites
0,101-01,record date: 2079-05-12 mercy care center mer...,"lipitor,synthroid",593,6
1,101-02,record date: 2079-06-23 mercy care center mer...,"atenolol,hydrochlorothiazide",442,10
2,101-03,record date: 2079-08-11 mercy care center mer...,"atenolol,ramipril,lipitor,long acting nitrate,...",608,41
3,105-02,record date: 2147-03-11 reason for consult: 6...,"lovastatin,fludrocortisone,celexa,proamatine,u...",1021,34
4,106-01,record date: 2083-11-17 northwest iowa health...,"vioxx,advil,ativan,zestril,diabetes medication...",754,21
...,...,...,...,...,...
345,394-04,record date: 2095-01-31 team 2 admission note...,"levofloxacin,lopressor,sotalol,prednisone,iron...",1970,113
346,395-02,record date: 2106-05-02 slhc emergency dept v...,"prednisone,antibiotics,enoxaparin",518,12
347,396-03,record date: 2078-10-25 chief complaint annua...,"novolog,aspirin,insulin aspart 70/30,ascorbic ...",1264,103
348,398-01,record date: 2105-12-15 mr. bryan comes in to...,"captopril,atenolol,aspirin,acetylsalicylic acid",313,18


In [19]:
df['tokenizer_length_entities'] = df.apply(lambda row: len(tokenizer.encode(str(row['entities']))) , axis=1)

In [20]:
df.sort_values('tokenizer_length_entities')

,filename,text,entities,tokenizer_length_text,tokenizer_length_entites,tokenizer_length_entities
241,299-05,record date: 2064-08-11 dcgh cancer center di...,NaN,406,1,1
189,258-04,record date: 2069-04-15 edvisit^49921666^schw...,NaN,525,1,1
101,182-02,"record date: 2128-04-08 name: doherty, daniel...",NaN,600,1,1
105,183-02,"record date: 2093-01-18 january 18, 2093 edwa...",NaN,499,1,1
316,366-01,record date: 2059-08-07 pcc emergency dept vi...,NaN,530,1,1
...,...,...,...,...,...,...
288,341-04,record date: 2097-12-15 sections of this note...,"anticoagulation,steroids,ivf,morphine,coumadin...",2883,226,226
324,371-05,record date: 2080-05-27 71f w/ mmps returns t...,"lidoderm,oxycodone,carvedilol,diovan,hydrochlo...",1665,234,234
289,341-05,record date: 2098-08-03 internal medicine g i...,"coumadin,vanc,ctx,clonidine,insulin glargine,m...",1968,236,236
112,191-02,record date: 2094-03-19 gsh internal medicine...,"rifampin,bactrim,o2,zofran,vanco,clinda,ns,nph...",2836,242,242


In [22]:
df.loc[323, 'entities']

'narcotics,zantac,ranitidine hcl,percocet,ativan,naproxen,nexium,lasix,furosemide,hydrochlorothiazide,levothyroxine sodium,loratadine,diovan,plavix,lipitor,carvedilol,oxycodone,penicillins,hctz,metfromin,estrogen,ibuprofen,caltrate and d,oxycodone+apap,lorazepam,miralax,polyethylene glycol,estrace,estradiol,esomeprazole,colace,docusate sodium,neurontin,gabapentin,fosamax,alendronate,valsartan,clopidogrel,metformin,calcium + d,calcium carbonate 1250 mg (500mg elem ca)/ vit d 200 iu,senna,sennosides,atorvastatin,corticosteroids,gadolinium chelates,gadolinuim,gadolinium chealtes'

In [56]:
print('texts bigger than 1024:', len(df.loc[df['tokenizer_length']>1024])/len(df))
print('texts smaller than 1024:', len(df.loc[df['tokenizer_length']<1024])/len(df))

texts bigger than 1024: 0.40285714285714286
texts smaller than 1024: 0.5942857142857143


In [36]:
print('texts bigger than 2024:', len(df.loc[df['tokenizer_length']>1500]))

texts bigger than 2024: 72


# Splitted with spacy



In [41]:
def text_preprocess(text):
    for iter in range(10):
      text = text.replace('\n\n', '\n').replace('\n ', '\n').replace(' \n', '\n').replace('\t', ' ')
    text_filtered = ''
    for char_index, char in enumerate(text):
        if char_index < (len(text)-1):
            if not(text[char_index + 1] == ' ' and char == ' '):
                text_filtered += char
    return text_filtered.lower()

In [52]:
for dataset_type in dataset_types:
    filenames = next(walk(my_paths[dataset_type]), (None, None, []))[2]
    # print(filenames[:20])
    filenames_ann = [filename for filename in filenames if 'ann' in filename]
    filenames_txt = [filename for filename in filenames if 'txt' in filename]
    filenames_ann.sort()
    filenames_txt.sort()
    filenames_ann_txt = list(zip(filenames_ann, filenames_txt))

    df_entities = pd.DataFrame()
    df_dataset = pd.DataFrame()
    for index, (filename_ann, filename_txt) in tqdm(enumerate(filenames_ann_txt)):
        # print('Current index:', index)value
        df_ann = pd.read_csv(my_paths[dataset_type] + filename_ann, sep='\t', names=['entity-event-context', 'classification-type', 'value'])
        df_entities = pd.DataFrame()
        # print(filename_ann)
        if (df_ann.apply(lambda row: row['value'] if row['entity-event-context'][0]== 'T' else '', axis=1).shape[0] > 0):
            df_entities['entities'] = df_ann.apply(lambda row: row['value'].lower() if row['entity-event-context'][0]== 'T' else '', axis=1)
            df_entities = df_entities[df_entities['entities'] != '']
            df_entities = df_entities.drop_duplicates(['entities'])
            entities =  ','.join(df_entities.entities)
        else:
            entities = ''

        with open(my_paths[dataset_type] + filename_txt, 'r') as file:
            text = ''.join(file.readlines())

        # text = ''.join(text).replace('\n', ' ').replace('\t', ' ')
        # text_filtered = ''
        # for char_index, char in enumerate(text):
        #     if char_index < (len(text)-1):
        #         if not(text[char_index + 1] == ' ' and char == ' '):
        #             text_filtered += char
        # text_filtered = text_filtered.lower()
        doc = nlp(text)
        max_lenght = 400
        text_slices = []
        sent_index = 0
        sents = [text_preprocess(sent.text) for sent in doc.sents]
        text_slice = ''
        for sent in sents:
            if len(tokenizer.encode(text_slice + sent)) > max_lenght:
                text_slices.append(text_slice)
                text_slice = sent
            else:
                text_slice += sent
        text_slices.append(text_slice)

        # print(entities)

        for sliced_text in  text_slices:
            filtered_entities = [entity for entity in entities.split(',') if entity in sliced_text]
            if len(filtered_entities) > 0:
                filtered_entities = ','.join(filtered_entities)
            else:
                filtered_entities = ''
            # filtered_entities = 
            row = pd.DataFrame({ 'filename': filename_ann[:-4], 'text':  sliced_text, 'entities': [filtered_entities] })
            df_dataset = pd.concat([df_dataset, row])
    df_dataset.to_csv('../data/trainingdata_v3/' + 'datasets/' + dataset_type + '_text_splitted'+ str(max_lenght) +'_entities.tsv', sep='\t')

350it [13:10,  2.26s/it]
50it [01:56,  2.33s/it]


In [37]:
with open('../data/trainingdata_v3/train/' + '359-02.txt', 'r') as file:
    text = ''.join(file.readlines())

# text = ''.join(text).replace('\n', ' ').replace('\t', ' ')
# text_filtered = ''
# for char_index, char in enumerate(text):
#     if char_index < (len(text)-1):
#         if not(text[char_index + 1] == ' ' and char == ' '):
#             text_filtered += char
# text_filtered = text_filtered.lower()
# print(text)
doc = nlp(text)
max_lenght = 100
text_ids_slices = []
sent_index = 0
sents = [text_preprocess(sent.text) for sent in doc.sents]
text_slice = ''
for sent in sents:
    if len(tokenizer.encode(text_slice + sent)) > max_lenght:
        text_ids_slices.append(tokenizer.encode(text_slice))
        text_slice = sent
    else:
        text_slice += sent
text_ids_slices.append(tokenizer.encode(text_slice))


['\nrecord date: 2091-11-22\n57 yo wf\ncc:\nhtn; tolerating toprol 150 mg/d, and lisinopril 20 mg\nhbpm about 145 sys and about 90 diastolic\nhx', ' htn found 10/89\npmhx htn\ndm 2,\nhlp\nhypothyroidism\nmeds\nmedications\nlisinopril 20 mg, 1 tablet(s) po qd : take as directed did not tak', '\ndyazide 1 capsule 50/25 po qd : take as directed\nlevothyroxine sodium 50mcg, 1 tablet(s) po qd : take as directed\nlipitor (atorvastatin) 20mg, 1 tablet(s) po qd : take 1 hour post supper or at bed time\nasa (children 81mg, 1 tablet(s) qd : take as directed\ntoprol xl (metoprolol succinate extended release) 100mg, 1.5 tablet(s) po qd', 'al', 'pnc ras', '\nfhx\ncad m ca no endo m with dm', 'sh', '\ntobacco neg etoh neg drugs neg', 'wor', ' detective marital div 1 c dom violence neg belts no/ advised to use seatbelts.', 'ros\nvision n', ' hearing nl fever neg\nheadache neg weakness neg numbness neg seizures neg\nchest pain neg', 'palpitatio', 'ne', ' syncope neg\ndyspnea neg wheezing neg cough neg

In [47]:
df_prova = pd.read_csv('../data/trainingdata_v3/' + 'datasets/train_text_splitted750_entities.tsv', sep='\t')

In [48]:
df_prova

,Unnamed: 0,filename,text,entities
0,0,101-01,\nrecord date: 2079-05-12\nmercy care center\n...,"lipitor,synthroid"
1,0,101-02,\nrecord date: 2079-06-23\nmercy care center\n...,"atenolol,hydrochlorothiazide"
2,0,101-03,\nrecord date: 2079-08-11mercy care cente\nmer...,"atenolol,ramipril,lipitor,long acting nitrate,..."
3,0,105-02,\nrecord date: 2147-03-11\nreason for consult:...,"lovastatin,fludrocortisone,celexa,proamatine,u..."
4,0,105-02,mr. tuttle is a 65 year old male with a three ...,NaN
...,...,...,...,...
691,0,396-03,\nrecord date: 2078-10-25chief complain\nannua...,"novolog,aspirin,insulin aspart 70/30,ascorbic ..."
692,0,396-03,"\ngu: no dysuria, hematuria.\nlmp: late 40s\nc...",NaN
693,0,398-01,\nrecord date: 2105-12-15mr. bryan comes in to...,"captopril,aspirin,acetylsalicylic acid"
694,0,399-02,\nrecord date: 2070-03-24\nneurology clinic fo...,"aspirin,trazodone,neurontin,dilaudid,antibiotics"


In [31]:
tokenizer.decode(text_ids_slices[0])

'\nrecord date: 2091-11-22\n57 yo wf\ncc:\nhtn; tolerating toprol 150 mg/d, and lisinopril 20 mg\nhbpm about 145 sys and about 90 diastolic\nhx'

In [34]:
tokenizer.decode(text_ids_slices[1])

' htn found 10/89\npmhx htn\ndm 2,\nhlp\nhypothyroidism\nmeds\nmedications\nlisinopril 20 mg, 1 tablet(s) po qd : take as directed did not tak'

In [27]:
# df = pd.read_csv('../data/trainingdata_v3/train/' + 'train' + '_text_splitted_entities.tsv', sep='\t')

# Splitted every N tokens

In [8]:
from transformers import AutoTokenizer
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
from tqdm import tqdm

In [12]:
for dataset_type in dataset_types:
    filenames = next(walk(my_paths[dataset_type]), (None, None, []))[2]
    # print(filenames[:20])
    filenames_ann = [filename for filename in filenames if 'ann' in filename]
    filenames_txt = [filename for filename in filenames if 'txt' in filename]
    filenames_ann.sort()
    filenames_txt.sort()
    filenames_ann_txt = list(zip(filenames_ann, filenames_txt))

    df_entities = pd.DataFrame()
    df_dataset = pd.DataFrame()
    for index, (filename_ann, filename_txt) in tqdm(enumerate(filenames_ann_txt)):
        # print('Current index:', index)value
        df_ann = pd.read_csv(my_paths[dataset_type] + filename_ann, sep='\t', names=['entity-event-context', 'classification-type', 'value'])
        df_entities = pd.DataFrame()
        # print(filename_ann)
        if (df_ann.apply(lambda row: row['value'] if row['entity-event-context'][0]== 'T' else '', axis=1).shape[0] > 0):
            df_entities['entities'] = df_ann.apply(lambda row: row['value'].lower() if row['entity-event-context'][0]== 'T' else '', axis=1)
            df_entities = df_entities[df_entities['entities'] != '']
            df_entities = df_entities.drop_duplicates(['entities'])
            entities =  ','.join(df_entities.entities)
        else:
            entities = ''

        with open(my_paths[dataset_type] + filename_txt, 'r') as file:
            text = file.readlines()

        text = ''.join(text).replace('\n', ' ').replace('\t', ' ')
        text_filtered = ''
        for char_index, char in enumerate(text):
            if char_index < (len(text)-1):
                if not(text[char_index + 1] == ' ' and char == ' '):
                    text_filtered += char
        text_filtered = text_filtered.lower()

        text_filtered_ids = tokenizer.encode(text_filtered)
        text_slices = []
        start_slice = 0
        max_lenght = 750
        for end_slice in range(max_lenght, len(text_filtered_ids), max_lenght):
            text_slices.append([start_slice, end_slice])
            start_slice = end_slice
        text_slices.append([start_slice, len(text_filtered_ids)])

        # print(entities)

        for start, end in  text_slices:
            sliced_text =  tokenizer.decode(text_filtered_ids[start:end])
            filtered_entities = [entity for entity in entities.split(',') if entity in sliced_text]
            if len(filtered_entities) > 0:
                filtered_entities = ','.join(filtered_entities)
            else:
                filtered_entities = ''
            # filtered_entities = 
            row = pd.DataFrame({ 'filename': filename_ann[:-4], 'text':  sliced_text, 'entities': [filtered_entities] })
            df_dataset = pd.concat([df_dataset, row])
    df_dataset.to_csv('../data/trainingdata_v3/datasets/' + dataset_type + '_text_splitted750_entities2.tsv', sep='\t')

350it [00:06, 57.32it/s]
50it [00:00, 52.85it/s]


Disposition


In [49]:
for dataset_type in dataset_types:
    filenames = next(walk(my_paths[dataset_type]), (None, None, []))[2]
    # print(filenames[:20])
    filenames_ann = [filename for filename in filenames if 'ann' in filename]
    filenames_txt = [filename for filename in filenames if 'txt' in filename]
    filenames_ann.sort()
    filenames_txt.sort()
    filenames_ann_txt = list(zip(filenames_ann, filenames_txt))

    df_dataset = pd.DataFrame()
    for index, (filename_ann, filename_txt) in enumerate(filenames_ann_txt):
        with open(my_paths[dataset_type] + filename_txt, 'r') as file:
            text = file.readlines()
        text = ''.join(text).replace('\n', ' ').replace('\t', ' ')
        text_filtered = ''
        for char_index, char in enumerate(text):
            if char_index < (len(text)-1):
                if not(text[char_index + 1] == ' ' and char == ' '):
                    text_filtered += char
        text_filtered = text_filtered.lower()
        # print('Current index:', index)value
        df_ann = pd.read_csv(my_paths[dataset_type] + filename_ann, sep='\t', names=['entity-event-context', 'classification-type', 'value'])
        df_entities = pd.DataFrame()

        # print(filename_ann)
        if (df_ann.apply(lambda row: row['value'] if row['entity-event-context'][0]== 'T' else '', axis=1).shape[0] > 0):
            df_entities['entity'] = df_ann.apply(lambda row: row['value'].lower() if row['entity-event-context'][0]== 'T' else '', axis=1)
            df_entities['disposition'] = df_ann.apply(lambda row: row['classification-type'].lower().split(' ')[0] if row['entity-event-context'][0]== 'T' else '', axis=1)
            df_entities = df_entities[df_entities['entity'] != '']
            df_entities = df_entities.drop_duplicates(['entity'])
            df_entities['filename'] = filename_ann[:-4]
            df_entities['text'] = text_filtered
            entitites =  ','.join(df_entities.entity)
            df_entities['output'] = df_entities.apply(lambda row: 'medication: ' + str(row['entity']) + '<SEPO>' + 'disposition: ' + str(row['disposition']), axis=1)
            df_entities = df_entities[['filename', 'text', 'output']]
        else:
            entitites = ''
        row = pd.DataFrame({ 'filename': filename_ann[:-4], 'text': [text_filtered], 'output': ['medication_list: ' + entitites]})
        df_dataset = pd.concat([df_dataset, df_entities, row], ignore_index=True)
    #     df_dataset = pd.concat([df_dataset, row], ignore_index=True)

    # # print(df_dataset.head())
    df_dataset.to_csv(my_paths[dataset_type] + dataset_type + '_entities_disposition.tsv', sep='\t')

In [50]:
df_dataset

,filename,text,output
0,100-01,record date: 2106-02-12 campbell orthopedic a...,medication: prozac<SEPO>disposition: nodisposi...
1,100-01,record date: 2106-02-12 campbell orthopedic a...,medication: cardizem<SEPO>disposition: nodispo...
2,100-01,record date: 2106-02-12 campbell orthopedic a...,medication: amaryl<SEPO>disposition: nodisposi...
3,100-01,record date: 2106-02-12 campbell orthopedic a...,medication: glucophage<SEPO>disposition: nodis...
4,100-01,record date: 2106-02-12 campbell orthopedic a...,"medication_list: prozac,cardizem,amaryl,glucop..."
...,...,...,...
793,397-03,record date: 2076-09-21 patient name: goldber...,medication: glyburide<SEPO>disposition: nodisp...
794,397-03,record date: 2076-09-21 patient name: goldber...,medication: codeine<SEPO>disposition: nodispos...
795,397-03,record date: 2076-09-21 patient name: goldber...,medication: ms contin<SEPO>disposition: nodisp...
796,397-03,record date: 2076-09-21 patient name: goldber...,medication: unasyn<SEPO>disposition: undetermined


In [ ]:
df_dataset['output'] = df_dataset.apply(lambda row: row['medication: ' + row['entity'] + '<SEPO>' + 'disposition: ' + row['disposition']], axis=1)